In [1]:
from reload_recursive import reload_recursive
import mri_data
import monai_training

In [2]:
from pathlib import Path
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data import file_manager as fm
from mri_data import utils
from monai_training import preprocess
from loguru import logger
import pyperclip

In [3]:
logger.remove()
dataroot = Path("/mnt/h/3Tpioneer_bids")

In [4]:
dataset_full = preprocess.DataSetProcesser.new_dataset(dataroot, fm.scan_3Tpioneer_bids, filters=[fm.filter_first_ses]).dataset
dataset_full.sort()
labeled = dict(subjects=[scan.subid for scan in dataset_full])

In [5]:
from collections import defaultdict
itksnap_commands = {}
for scan in dataset_full:
    images = [scan.root/"flair.nii.gz", scan.root/"t1.nii.gz"]
    itksnap_commands[scan.subid] = utils.open_itksnap_workspace_cmd(images, win=True)

In [41]:
cmd = itksnap_commands['1247']
print(cmd)
pyperclip.copy(cmd)

itksnap -g H:/3Tpioneer_bids/sub-ms1247/ses-20190905/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1247/ses-20190905/t1.nii.gz


In [20]:
dataset_proc = preprocess.DataSetProcesser.new_dataset(dataroot, fm.scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
dataset_proc.prepare_labels("pituitary", suffix_list=["CH", "ED", "DT"])
pituitary_subs = [scan.subid for scan in dataset_proc.dataset]

100%|██████████| 575/575 [00:04<00:00, 135.34it/s]


In [23]:
dataset_proc = preprocess.DataSetProcesser.new_dataset(dataroot, fm.scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
dataset_proc.prepare_labels("pineal", suffix_list=["CH", "SRS"])
pineal_subs = [scan.subid for scan in dataset_proc.dataset]

100%|██████████| 575/575 [00:04<00:00, 122.89it/s]


In [24]:
dataset_proc = preprocess.DataSetProcesser.new_dataset(dataroot, fm.scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
dataset_proc.prepare_labels("choroid_t1_flair", suffix_list=["CH", "ED", "DT"])
choroid_subs = [scan.subid for scan in dataset_proc.dataset]

100%|██████████| 575/575 [00:04<00:00, 121.05it/s]


In [41]:
labeled['pituitary'] = [True if scan.subid in pituitary_subs else "" for scan in dataset_full]
labeled['pineal'] = [True if scan.subid in pineal_subs else "" for scan in dataset_full]
labeled['choroid'] = [True if scan.subid in choroid_subs else "" for scan in dataset_full]

In [42]:
import csv

with open("labeled_subjects.csv", 'w') as f:
    writer = csv.DictWriter(f, labeled.keys())
    writer.writeheader()
    for i in range(len(labeled['subjects'])):
        writer.writerow({k: labeled[k][i] for k in labeled})